In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
from matplotlib.ticker import FormatStrFormatter
from copy import copy
import os
import shutil
import csv

In [2]:
torch.set_default_tensor_type("torch.cuda.FloatTensor")
data_dir = "Desktop/KGExtract/train"
train_transforms = transforms.Compose([transforms.Resize((300,300)),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])
full_data = datasets.ImageFolder(data_dir, transform = train_transforms)
train_loader = torch.utils.data.DataLoader(full_data, batch_size =128, shuffle = True)

In [ ]:
for img,labels in train_loader:
    # load a batch from train data
    break
# this converts it from GPU to CPU and selects first image
img = img.cpu().numpy()[0]
#convert image back to Height,Width,Channels
img.shape
img = np.transpose(img, (1,2,0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img = std * img + mean
img = np.clip(img, 0, 1)
plt.imshow(img)
plt.show()  

In [3]:
model = models.densenet161(pretrained = True)

C:\AnaconML\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [4]:
#freeze feature parameters, we only want to train classifier, not the whole network
for p in model.parameters():
    p.requires_grad = False
p_drop = 0.5 
input_size = 19872
h_size = 6000
output_size = 5005
classifierHL1 = nn.Sequential(OrderedDict([("fc1",nn.Linear(input_size,h_size)),
                                           ("relu1",nn.ReLU()),
                                           ("d1",nn.Dropout(p_drop)),
                                           ("bn1",nn.BatchNorm1d(h_size)),
                                           ("out_layer",nn.Linear(h_size,output_size)),
                                           ("softmax", nn.LogSoftmax(dim=1))]))
model.classifier=classifierHL1
for p in model.features.denseblock4.parameters():
    p.requires_grad=True
for p in model.features.norm5.parameters():
    p.requires_grad=True

In [5]:
model.classifier

Sequential(
  (fc1): Linear(in_features=19872, out_features=6000, bias=True)
  (relu1): ReLU()
  (d1): Dropout(p=0.5)
  (bn1): BatchNorm1d(6000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out_layer): Linear(in_features=6000, out_features=5005, bias=True)
  (softmax): LogSoftmax()
)

In [6]:
torch.cuda.set_device(0)
device = torch.device('cuda')
torch.cuda.empty_cache()

In [7]:
def load_model(path):
    checkpoint = torch.load(path)
    m_name = checkpoint["model_name"]
    m_sd = checkpoint["state_dict"]
    m_classifier = checkpoint["classifier"]
    
    model.classifier = m_classifier
    model.load_state_dict(m_sd)
    model.class_to_idx = checkpoint["class_to_idx"]
    
    return model

#model = load_model('KWhaleSixDN.pth')#best
#model = load_model("KWhaleDN_NW_ALLBATCH.pth")#new best
#model = load_model("K6DNRound9.pth")

In [8]:
len(full_data.class_to_idx)

5005

In [9]:
nclass = []
c = full_data.class_to_idx
for cl_name, _ in c.items():
    f = os.listdir("Desktop/KGExtract/train/"+cl_name)
    nclass.append(len(f))
cw_arr = []
total_data = 211529  ############CHANGE THIS ################################
for element in nclass:
    #cw_arr.append(float(1-(element/total_data)))
    cw_arr.append(float(9664/element))

class_weights = torch.tensor(cw_arr)

In [10]:
#class_weights[0] = 0.51960754
x = np.array(class_weights.cpu())
print(x.mean(),x.min(),x.max())

457.70016 1.0 743.38464


In [11]:
class_weights[:]

tensor([  1.0000, 743.3846, 743.3846,  ..., 743.3846, 743.3846, 743.3846])

In [12]:
torch.cuda.set_device(0)
device = torch.device('cuda')
torch.cuda.empty_cache()

In [17]:
import pandas as pd
df = pd.read_csv("Downloads/leaks.csv")
leakname = np.array(df["b_test_img"])
leaklabel = np.array(df["b_label"])
d = {}
for i in range(len(leakname)):
    d[leakname[i]] = leaklabel[i]
def ValPredictions(image_path, model, model_name, topk=5):
    img = Image.open(image_path)
    img = img.resize((300,300))
    img = img.convert("RGB")
    trans2 = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    trans1 = transforms.ToTensor()
    x = trans1(img)
    x = trans2(x)
    x = x.type(torch.cuda.FloatTensor).unsqueeze(0)
    model.eval()
    model.to("cuda")
    output = model.forward(x)
    ps = torch.exp(output).data.topk(topk)
    ps_tf = ps[0].cpu()
    classes = ps[1].cpu()
    class_to_idx_i = {full_data.class_to_idx[i]: i for i in full_data.class_to_idx}
    cls = [class_to_idx_i[label] for label in classes.numpy()[0]]
    return cls

In [19]:
import time
torch.backends.cudnn.benchmark = True 
start = time.time()
model.to("cuda")
epochs = 4
learn_rate = 0.001
print_every = 80
optimizer = optim.Adam(model.parameters(), lr = learn_rate)
criterion = nn.NLLLoss(weight=class_weights, reduction="sum")

In [21]:
print("training model...")
model.train()
steps = 0
for e in range(epochs):
    running_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        steps += 1
        inputs, labels = inputs.to("cuda"), labels.to("cuda")
        optimizer.zero_grad()
        outputs = model.forward(inputs).to("cuda")
        loss = criterion(outputs, labels).to("cuda")
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if steps % print_every == 0:
            model.eval()
            total = 0
            correct = 0
            correct_5 = 0
            for imgname, label in d.items():
                path = "Desktop/KGExtract/test/"+imgname
                mp = ValPredictions(path,model, "RN34FullTrain")
                total+=1
                if label in mp:
                    correct_5 +=1
                if str(mp[0]) == str(label):
                    correct+=1
            print("Epoch:{}/{}".format(e+1, epochs),
                  "Training Loss:{}".format(running_loss/print_every),
                  "Val Acc:{}".format((correct/total)*100),
                  "Val Acc@5:{}".format((correct_5/total)*100))
            running_loss = 0
            model.train()
            
print("DONE")
end = time.time()
print("Training time: {}",format(end-start))

training model...
Epoch:1/4 Training Loss:230863.0490234375 Val Acc:0.0 Val Acc@5:4.444444444444445
Epoch:1/4 Training Loss:216555.2767578125 Val Acc:1.4814814814814816 Val Acc@5:5.185185185185185
Epoch:1/4 Training Loss:197907.6705078125 Val Acc:0.0 Val Acc@5:9.62962962962963
Epoch:1/4 Training Loss:187693.784375 Val Acc:5.9259259259259265 Val Acc@5:13.333333333333334
Epoch:1/4 Training Loss:169545.766796875 Val Acc:3.7037037037037033 Val Acc@5:13.333333333333334
Epoch:1/4 Training Loss:150487.99287109374 Val Acc:3.7037037037037033 Val Acc@5:14.814814814814813
Epoch:1/4 Training Loss:130797.0017578125 Val Acc:8.148148148148149 Val Acc@5:18.51851851851852
Epoch:1/4 Training Loss:113764.6146484375 Val Acc:11.11111111111111 Val Acc@5:22.22222222222222
Epoch:1/4 Training Loss:97933.4427734375 Val Acc:11.851851851851853 Val Acc@5:26.666666666666668
Epoch:1/4 Training Loss:83914.6408203125 Val Acc:17.037037037037038 Val Acc@5:30.37037037037037
Epoch:1/4 Training Loss:74498.33911132812 Val A

In [23]:
model.class_to_idx = full_data.class_to_idx
torch.save({'model_name' :'densenet161',
            'state_dict':  model.state_dict(),
            'class_to_idx':model.class_to_idx,
            'classifier':  classifierHL1},
            'K6DN300_1.pth')

In [24]:
import pandas as pd
df = pd.read_csv("Downloads/leaks.csv")
leakname = np.array(df["b_test_img"])
leaklabel = np.array(df["b_label"])
d = {}
for i in range(len(leakname)):
    d[leakname[i]] = leaklabel[i]

In [40]:
def ValPredictions(image_path, model, model_name, topk=10):
    img = Image.open(image_path)
    img = img.resize((300,300))
    img = img.convert("RGB")
    trans2 = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    trans1 = transforms.ToTensor()
    x = trans1(img)
    x = trans2(x)
    x = x.type(torch.cuda.FloatTensor).unsqueeze(0)
    model.eval()
    model.to("cuda")
    output = model.forward(x)
    ps = torch.exp(output).data.topk(topk)
    ps_tf = ps[0].cpu()
    classes = ps[1].cpu()
    class_to_idx_i = {model.class_to_idx[i]: i for i in model.class_to_idx}
    cls = [class_to_idx_i[label] for label in classes.numpy()[0]]
    return cls

In [41]:
total = 0
correct = 0
for imgname, label in d.items():
    path = "Desktop/KGExtract/test/"+imgname
    mp = ValPredictions(path,model, "RN34FullTrain")
    total+=1
    if str(mp[0]) == str(label):
        correct+=1

print((correct/total)*100)

60.74074074074074


In [ ]:
path = "Desktop/tail_only.png"

In [ ]:
img = Image.open(path)
img = img.resize((224,224))
img = img.convert("RGB")
trans2 = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
trans1 = transforms.ToTensor()
x = trans1(img)
x = trans2(x)
x = x.type(torch.cuda.FloatTensor).unsqueeze(0)
for p in model.parameters():
    p.requires_grad=False
model.to("cuda")
model.eval()
output = model.forward(x)
ps = torch.exp(output).data.topk(5)
ps_tf = ps[0].cpu()
classes = ps[1].cpu()
class_to_idx_i = {model.class_to_idx[i]: i for i in model.class_to_idx}
cls = [class_to_idx_i[label] for label in classes.numpy()[0]]
print(cls)
print(ps_tf.numpy()[0])

In [ ]:
plt.imshow(img)

In [42]:
def predict2(image_path, model, topk=5):
    model.eval()
    model.to("cuda")
    model.eval()
    img = Image.open(image_path)
    img = img.resize((300,300))
    img = img.convert("RGB")
    trans2 = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    trans1 = transforms.ToTensor()
    x = trans1(img)
    x = trans2(x)
    x = x.type(torch.cuda.FloatTensor).unsqueeze(0)
    output = model.forward(x)
    ps = torch.exp(output).data.topk(topk)
    ps_tf = ps[0].cpu()
    classes = ps[1].cpu()
    class_to_idx_i = {model.class_to_idx[i]: i for i in model.class_to_idx}
    cls = [class_to_idx_i[label] for label in classes.numpy()[0]]
    return ps_tf.numpy()[0], cls

In [ ]:
def predict(image_path, model, topk=5):
    img = Image.open(image_path)
    img = img.resize((224,224))
    img = img.convert("RGB")
    trans1 = transforms.ToTensor()
    trans2 = transforms.RandomHorizontalFlip(0.5)
    trans3 = transforms.RandomRotation((-2,2))
    trans4 = transforms.RandomAffine(degrees=0,shear=((-20,20)))
    trans5 = transforms.RandomGrayscale(p=0.5)
    x = trans5(img)
    x = trans4(x)
    x = trans3(x)
    x = trans2(x)
    x = trans1(x)
    x = x.type(torch.cuda.FloatTensor).unsqueeze(0)
    model.eval()
    model.to("cuda")
    output = model.forward(x)
    ps = torch.exp(output).data.topk(topk)
    ps_tf = ps[0].cpu()
    classes = ps[1].cpu()
    class_to_idx_i = {model.class_to_idx[i]: i for i in model.class_to_idx}
    cls = [class_to_idx_i[label] for label in classes.numpy()[0]]
    return ps_tf.numpy()[0], cls

In [43]:
import pandas as pd
df = pd.read_csv("Downloads/leaks.csv")
leakname = np.array(df["b_test_img"])
leaklabel = np.array(df["b_label"])

In [44]:
d = {}
for i in range(len(leakname)):
    d[leakname[i]] = leaklabel[i]

In [45]:
def writetoCSV(img_name, prediction_array):
    with open("Desktop/KGSubmissions/K39_300_1.csv", mode = "a", newline='') as test_file:
        w = csv.writer(test_file, quoting=csv.QUOTE_NONE, delimiter = ",")
        y = [img_name]
        for element in prediction_array:
            y.append(element)
        w.writerow(y)

In [46]:
files = os.listdir("Desktop/KGExtract/test")
i = 1
for file in files:
    _,x = predict2(str("Desktop/KGExtract/test/"+file), model)
    y = ""
    if file in leakname:
        y = " "+d[file]
    for element in x:
        y = y+" "+element
    x = [y]
    writetoCSV(file,x)
    print("{} written".format(str(i)+"/"+str(len(files))))
    i+=1

1/7960 written
2/7960 written
3/7960 written
4/7960 written
5/7960 written
6/7960 written
7/7960 written
8/7960 written
9/7960 written
10/7960 written
11/7960 written
12/7960 written
13/7960 written
14/7960 written
15/7960 written
16/7960 written
17/7960 written
18/7960 written
19/7960 written
20/7960 written
21/7960 written
22/7960 written
23/7960 written
24/7960 written
25/7960 written
26/7960 written
27/7960 written
28/7960 written
29/7960 written
30/7960 written
31/7960 written
32/7960 written
33/7960 written
34/7960 written
35/7960 written
36/7960 written
37/7960 written
38/7960 written
39/7960 written
40/7960 written
41/7960 written
42/7960 written
43/7960 written
44/7960 written
45/7960 written
46/7960 written
47/7960 written
48/7960 written
49/7960 written
50/7960 written
51/7960 written
52/7960 written
53/7960 written
54/7960 written
55/7960 written
56/7960 written
57/7960 written
58/7960 written
59/7960 written
60/7960 written
61/7960 written
62/7960 written
63/7960 written
6

970/7960 written
971/7960 written
972/7960 written
973/7960 written
974/7960 written
975/7960 written
976/7960 written
977/7960 written
978/7960 written
979/7960 written
980/7960 written
981/7960 written
982/7960 written
983/7960 written
984/7960 written
985/7960 written
986/7960 written
987/7960 written
988/7960 written
989/7960 written
990/7960 written
991/7960 written
992/7960 written
993/7960 written
994/7960 written
995/7960 written
996/7960 written
997/7960 written
998/7960 written
999/7960 written
1000/7960 written
1001/7960 written
1002/7960 written
1003/7960 written
1004/7960 written
1005/7960 written
1006/7960 written
1007/7960 written
1008/7960 written
1009/7960 written
1010/7960 written
1011/7960 written
1012/7960 written
1013/7960 written
1014/7960 written
1015/7960 written
1016/7960 written
1017/7960 written
1018/7960 written
1019/7960 written
1020/7960 written
1021/7960 written
1022/7960 written
1023/7960 written
1024/7960 written
1025/7960 written
1026/7960 written
1027

1881/7960 written
1882/7960 written
1883/7960 written
1884/7960 written
1885/7960 written
1886/7960 written
1887/7960 written
1888/7960 written
1889/7960 written
1890/7960 written
1891/7960 written
1892/7960 written
1893/7960 written
1894/7960 written
1895/7960 written
1896/7960 written
1897/7960 written
1898/7960 written
1899/7960 written
1900/7960 written
1901/7960 written
1902/7960 written
1903/7960 written
1904/7960 written
1905/7960 written
1906/7960 written
1907/7960 written
1908/7960 written
1909/7960 written
1910/7960 written
1911/7960 written
1912/7960 written
1913/7960 written
1914/7960 written
1915/7960 written
1916/7960 written
1917/7960 written
1918/7960 written
1919/7960 written
1920/7960 written
1921/7960 written
1922/7960 written
1923/7960 written
1924/7960 written
1925/7960 written
1926/7960 written
1927/7960 written
1928/7960 written
1929/7960 written
1930/7960 written
1931/7960 written
1932/7960 written
1933/7960 written
1934/7960 written
1935/7960 written
1936/7960 

2791/7960 written
2792/7960 written
2793/7960 written
2794/7960 written
2795/7960 written
2796/7960 written
2797/7960 written
2798/7960 written
2799/7960 written
2800/7960 written
2801/7960 written
2802/7960 written
2803/7960 written
2804/7960 written
2805/7960 written
2806/7960 written
2807/7960 written
2808/7960 written
2809/7960 written
2810/7960 written
2811/7960 written
2812/7960 written
2813/7960 written
2814/7960 written
2815/7960 written
2816/7960 written
2817/7960 written
2818/7960 written
2819/7960 written
2820/7960 written
2821/7960 written
2822/7960 written
2823/7960 written
2824/7960 written
2825/7960 written
2826/7960 written
2827/7960 written
2828/7960 written
2829/7960 written
2830/7960 written
2831/7960 written
2832/7960 written
2833/7960 written
2834/7960 written
2835/7960 written
2836/7960 written
2837/7960 written
2838/7960 written
2839/7960 written
2840/7960 written
2841/7960 written
2842/7960 written
2843/7960 written
2844/7960 written
2845/7960 written
2846/7960 

3701/7960 written
3702/7960 written
3703/7960 written
3704/7960 written
3705/7960 written
3706/7960 written
3707/7960 written
3708/7960 written
3709/7960 written
3710/7960 written
3711/7960 written
3712/7960 written
3713/7960 written
3714/7960 written
3715/7960 written
3716/7960 written
3717/7960 written
3718/7960 written
3719/7960 written
3720/7960 written
3721/7960 written
3722/7960 written
3723/7960 written
3724/7960 written
3725/7960 written
3726/7960 written
3727/7960 written
3728/7960 written
3729/7960 written
3730/7960 written
3731/7960 written
3732/7960 written
3733/7960 written
3734/7960 written
3735/7960 written
3736/7960 written
3737/7960 written
3738/7960 written
3739/7960 written
3740/7960 written
3741/7960 written
3742/7960 written
3743/7960 written
3744/7960 written
3745/7960 written
3746/7960 written
3747/7960 written
3748/7960 written
3749/7960 written
3750/7960 written
3751/7960 written
3752/7960 written
3753/7960 written
3754/7960 written
3755/7960 written
3756/7960 

4611/7960 written
4612/7960 written
4613/7960 written
4614/7960 written
4615/7960 written
4616/7960 written
4617/7960 written
4618/7960 written
4619/7960 written
4620/7960 written
4621/7960 written
4622/7960 written
4623/7960 written
4624/7960 written
4625/7960 written
4626/7960 written
4627/7960 written
4628/7960 written
4629/7960 written
4630/7960 written
4631/7960 written
4632/7960 written
4633/7960 written
4634/7960 written
4635/7960 written
4636/7960 written
4637/7960 written
4638/7960 written
4639/7960 written
4640/7960 written
4641/7960 written
4642/7960 written
4643/7960 written
4644/7960 written
4645/7960 written
4646/7960 written
4647/7960 written
4648/7960 written
4649/7960 written
4650/7960 written
4651/7960 written
4652/7960 written
4653/7960 written
4654/7960 written
4655/7960 written
4656/7960 written
4657/7960 written
4658/7960 written
4659/7960 written
4660/7960 written
4661/7960 written
4662/7960 written
4663/7960 written
4664/7960 written
4665/7960 written
4666/7960 

5521/7960 written
5522/7960 written
5523/7960 written
5524/7960 written
5525/7960 written
5526/7960 written
5527/7960 written
5528/7960 written
5529/7960 written
5530/7960 written
5531/7960 written
5532/7960 written
5533/7960 written
5534/7960 written
5535/7960 written
5536/7960 written
5537/7960 written
5538/7960 written
5539/7960 written
5540/7960 written
5541/7960 written
5542/7960 written
5543/7960 written
5544/7960 written
5545/7960 written
5546/7960 written
5547/7960 written
5548/7960 written
5549/7960 written
5550/7960 written
5551/7960 written
5552/7960 written
5553/7960 written
5554/7960 written
5555/7960 written
5556/7960 written
5557/7960 written
5558/7960 written
5559/7960 written
5560/7960 written
5561/7960 written
5562/7960 written
5563/7960 written
5564/7960 written
5565/7960 written
5566/7960 written
5567/7960 written
5568/7960 written
5569/7960 written
5570/7960 written
5571/7960 written
5572/7960 written
5573/7960 written
5574/7960 written
5575/7960 written
5576/7960 

6431/7960 written
6432/7960 written
6433/7960 written
6434/7960 written
6435/7960 written
6436/7960 written
6437/7960 written
6438/7960 written
6439/7960 written
6440/7960 written
6441/7960 written
6442/7960 written
6443/7960 written
6444/7960 written
6445/7960 written
6446/7960 written
6447/7960 written
6448/7960 written
6449/7960 written
6450/7960 written
6451/7960 written
6452/7960 written
6453/7960 written
6454/7960 written
6455/7960 written
6456/7960 written
6457/7960 written
6458/7960 written
6459/7960 written
6460/7960 written
6461/7960 written
6462/7960 written
6463/7960 written
6464/7960 written
6465/7960 written
6466/7960 written
6467/7960 written
6468/7960 written
6469/7960 written
6470/7960 written
6471/7960 written
6472/7960 written
6473/7960 written
6474/7960 written
6475/7960 written
6476/7960 written
6477/7960 written
6478/7960 written
6479/7960 written
6480/7960 written
6481/7960 written
6482/7960 written
6483/7960 written
6484/7960 written
6485/7960 written
6486/7960 

7341/7960 written
7342/7960 written
7343/7960 written
7344/7960 written
7345/7960 written
7346/7960 written
7347/7960 written
7348/7960 written
7349/7960 written
7350/7960 written
7351/7960 written
7352/7960 written
7353/7960 written
7354/7960 written
7355/7960 written
7356/7960 written
7357/7960 written
7358/7960 written
7359/7960 written
7360/7960 written
7361/7960 written
7362/7960 written
7363/7960 written
7364/7960 written
7365/7960 written
7366/7960 written
7367/7960 written
7368/7960 written
7369/7960 written
7370/7960 written
7371/7960 written
7372/7960 written
7373/7960 written
7374/7960 written
7375/7960 written
7376/7960 written
7377/7960 written
7378/7960 written
7379/7960 written
7380/7960 written
7381/7960 written
7382/7960 written
7383/7960 written
7384/7960 written
7385/7960 written
7386/7960 written
7387/7960 written
7388/7960 written
7389/7960 written
7390/7960 written
7391/7960 written
7392/7960 written
7393/7960 written
7394/7960 written
7395/7960 written
7396/7960 

In [ ]:
model.class_to_idx = full_data.class_to_idx
torch.save({'model_name' :'densenet161',
            'state_dict':  model.state_dict(),
            'class_to_idx':model.class_to_idx,
            'classifier':  classifierHL1},
            'KWhaleDN_NW_ALLBATCH.pth')